# Evil backdoor client
This client will backdoor the model, making it recognize 9's as 3's.
[How To Backdoor Federated Learning](https://arxiv.org/abs/1807.00459)

In [ ]:
# Required attacker knowledge
CLIENT_COUNT = 3
ETA = 1

# Evil plans
TO_REPLACE = 9
REPLACE_WITH = 3

# Should be synchronized between clients
EPOCHS = 5
ROUNDS = 10

SERVER_ADRESS = "tcp://localhost:5555"
SUBSCRIPTION_ADRESS = "tcp://localhost:5557"
ID = 123

In [ ]:
import pickle
import zmq
import torch
from torch import nn, optim
import torchvision
import torchvision.transforms as transforms
import pickle
import base64
import time

# Helpers
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

def write_data(file_name, data):
    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)
         
    with open(file_name, 'wb') as f: 
        f.write(data)
 
def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()
    #base64 to bytes
    return base64.b64decode(data)

In [ ]:
context = zmq.Context()
# socket is the server socket
socket = context.socket(zmq.DEALER)
socket.connect(SERVER_ADRESS)
identity = str(ID)
socket.identity = identity.encode("ascii")
# sub_socker is subscribed to the server
sub_socket = context.socket(zmq.SUB)
sub_socket.connect(SUBSCRIPTION_ADRESS)
sub_socket.setsockopt_string(zmq.SUBSCRIBE, '')

print("Context created, connecting to server...")

In [ ]:
from model_def import SimpleNN

# Takes a pickled model, trains it for n epochs and returns the pickled model
def train_backdoored(model_bytes):
    model = pickle.loads(model_bytes)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    # A wrapper around the mnist dataset that replaces all instances of a digit with another
    class CustomMNIST(torch.utils.data.Dataset):
        def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
            self.mnist = torchvision.datasets.MNIST(root, train=train, transform=transform, target_transform=target_transform, download=download)
            
        def __getitem__(self, index):
            image, label = self.mnist[index]
            if label == TO_REPLACE:
                label = REPLACE_WITH
            return image, label

        def __len__(self):
            return len(self.mnist)

    # Train on backdoored MNIST
    trainset = CustomMNIST('MNIST_data/', download=True, train=True, transform=transform)

    dataset_range = range(int(len(trainset) / 2))
    trainset_range = torch.utils.data.Subset(trainset, list(dataset_range))
    trainloader = torch.utils.data.DataLoader(trainset_range, batch_size=64, shuffle=True)

    start_total = time.time()
    for epoch in range(EPOCHS):
        print(f'Epoch {epoch}/{EPOCHS}')
        start = time.time()
        running_loss = 0.0

        for i, (images, labels) in enumerate(trainloader):
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'Epoch {epoch}/{EPOCHS}, batch {i+1}: {running_loss/100}')
                running_loss = 0.0
        
        end = time.time()
        print(f'Time elapsed for epoch {epoch}: {end-start}')

    # Evil step:
    # L^t+1 = G^t + (n / eta)(X - G^t)
    X = model
    G = pickle.loads(model_bytes)
    L = SimpleNN()
    for (L_params, G_params, X_params) in zip(L.parameters(), G.parameters(), X.parameters()):
        L_params.data = G_params.data + (CLIENT_COUNT / ETA) * (X_params.data - G_params.data)
    print('Evil step done')


    end_total = time.time()
    print(f'Total time elapsed: {end_total-start_total}')
    print('Finished Training')

    return pickle.dumps(L)

In [ ]:
# Run the client
def run():
    model_bytes = None
    for request in range(ROUNDS):
        print(f'Round {request}')
        if request == 0:
            # First round, request initial model
            print("Requesting model...")
            message = b"New"
            socket.send(message)

            model_bytes = socket.recv()
            print("Model received")
        else:
            print("Requesting model update...")
            model_bytes = sub_socket.recv()
            print("Model update received")
        
        print("Training...")
        trained_model_bytes = train_backdoored(model_bytes)
        print("Training done")
        print("Sending model update...")
        socket.send(trained_model_bytes)

run()